#### Analysieren von Amazon Review Daten.
In diesem Skript ein Datensatz bestehend aus Review Daten des Versandhändlers Amazon analysiert. 

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark
import time
import math
from pyspark.sql.functions import col

bei .master ("local[*]") einsetzen, oder ein master + slaves starten (steigert Performance um das 4-fache)

In [2]:
# Session anpassen
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Amazon Reviews") \
    .getOrCreate()
# .master("spark://192.168.1.247:7077") \

23/01/12 18:06:01 WARN Utils: Your hostname, Pascals-MacBook-Pro-3.local resolves to a loopback address: 127.0.0.1; using 193.197.227.148 instead (on interface en0)
23/01/12 18:06:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/12 18:06:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Lade JSON Datei
df = spark.read.json(r'/Users/pascal/Desktop/Big Data/All_Beauty.json')

# Entferne Spalten
df = df.drop('verified', 'reviewTime',"reviewerID", "asin","reviewerName","summary","unixReviewTime","vote", "style", "image")

# replacing any null values in the 'reviewText' column with an empty space
df = df.fillna(" ", subset=["reviewText"])

df = df.withColumn("Textlänge_integer", pyspark.sql.functions.length('reviewText'))

# casting the "Textlänge_integer" column to a double data type and renaming it to "Textlänge"
df = df.withColumn("Textlänge", col("Textlänge_integer").cast("double"))

df = df.drop("reviewText", "Textlänge_integer")

# replacing any null values in the "Textlänge" column with 0
df.fillna(0, subset=["Textlänge"])

# Dataframe in RDD konvertieren.
partitionen = 10
data = df.rdd.repartition(partitionen)

Korrelation eigenständig berechnen

In [4]:
# berechnet den Mittelwert einer bestimmten Spalte
def calculate_mean(data, column):
    return data.map(lambda x: x[column]).sum() / n

# berechnet die Standardabweichung einer bestimmten Spalte
def calculate_std(data, column, mean):
    return math.sqrt(data.map(lambda x: (x[column] - mean)**2).sum() / n)

# berechnet die Kovarianz zweier Spalten in den Daten
def calculate_cov(data, column1, column2, mean1, mean2):
    return data.map(lambda x: (x[column1] - mean1) * (x[column2] - mean2)).sum() / n

start_time = time.time()
n = data.count()
mean_col1 = calculate_mean(data, 0)
mean_col2 = calculate_mean(data, 1)
std_col1 = calculate_std(data, 0, mean_col1)
std_col2 = calculate_std(data, 1, mean_col2)
covXY = calculate_cov(data, 0, 1, mean_col1, mean_col2)
correlationOwnCalculation = covXY / (std_col1 * std_col2)
executionTimeOwnCalculation = time.time() - start_time

print("Korrelation: ", correlationOwnCalculation)
print(f"Execution time in seconds: {time.time() - start_time}")

<class 'float'>
Korrelation:  -0.020101181997881704
Execution time in seconds: 2.837958812713623


# Pyspark.SQL Korrelation

In [5]:
#Korrelation berechnen
start_time = time.time()
correlationPysparkSql = pyspark.sql.DataFrameStatFunctions(df).corr("overall", "Textlänge")
executionTimePysparkSql = time.time() - start_time
print("Korrelation: ", correlationPysparkSql)
print(f"Execution time in seconds: {time.time() - start_time}")

Korrelation:  -0.020101181997884847
Execution time in seconds: 0.5301339626312256


# Pandas Korrelation

In [6]:
#DataFrame in ein Pandas_Dataframe umwandeln
start_time = time.time()
data = df.toPandas()
print(f"Execution time in seconds: {time.time() - start_time}")

Execution time in seconds: 1.3379290103912354


In [13]:
#Korrelation mit Panda-DataFrame berechnen
start_time = time.time()
correlationPandas = data["overall"].corr(data["Textlänge"])
executionTimePandas = time.time() - start_time
print(f"Execution time in seconds: {time.time() - start_time}")

-0.020101181997883685
Execution time in seconds: 0.009268999099731445


# Code um Duplikate zu bilden

In [8]:
# df = spark.read.json(r'C:\Users\klass\data\all_beauty.json')
#
# # Inhalt duplizieren
# df = df.union(df)
# df = df.coalesce(1)
#
# # Inhalt in eine Datei schreiben
# df.write.json("duplicated_large_file.json", mode="overwrite")